In [6]:
from pathlib import Path
from dnafiber.ui.utils import get_image_cacheless
import cv2
from tqdm.auto import tqdm

In [14]:
root = Path("C:/Users/cleme/OneDrive/Documents/postdoc/data/DNAI_S1/S1-labeling")
export_root = Path("C:/Users/cleme/OneDrive/Documents/postdoc/data/DNAI_S1/export")
export_root.mkdir(parents=True, exist_ok=True)
all_files = list(root.rglob("*.czi"))

for file in tqdm(all_files):
    img = get_image_cacheless(file, reverse_channel=False)
    fx = 0.5
    fy = 0.5
    img_resized = cv2.resize(img, (0, 0), fx=fx, fy=fy, interpolation=cv2.INTER_AREA)
    if "04" in file.stem or "05" in file.stem or "06" in file.stem:
        folder = export_root / 'test'
    else:
        folder = export_root / 'train'
    folder.mkdir(parents=True, exist_ok=True)
    # Split into multiples crop of 2048x2048
    h, w = img_resized.shape[:2]
    crop_size = 2048
    for i in range(0, h, crop_size):
        for j in range(0, w, crop_size):
            crop = img_resized[i:i+crop_size, j:j+crop_size]
            # More than 50% of the crop should be non black
            if (crop > 0).sum() > 0.5 * crop_size * crop_size:
                if crop.shape[0] == crop_size and crop.shape[1] == crop_size:
                    
                    crop_filename = folder / f"{file.stem}_crop_{i}_{j}.png"
                    cv2.imwrite(str(crop_filename), crop[:,:, ::-1])  # Convert RGB to BGR for OpenCV
        


  0%|          | 0/34 [00:00<?, ?it/s]